# Silver Analytics - Acidentes Aéreos (One Big Table)

Análise avançada dos dados tratados da camada SILVER armazenados no banco PostgreSQL na tabela `silver.acd`.

---

## 1. Introdução

Este notebook conecta ao banco de dados PostgreSQL, carrega a tabela `silver.acd` e gera análises avançadas com gráficos de barras, pizza, temporal e correlações usando Pandas.

### 1.1 Configuração do ambiente (Pandas/SQL)
- **Objetivo**:
    - Preparar ambiente visual e uso de Pandas para análise.
- **Ações:**
    - Aplica tema gráfico.
- **Saída:**
    - Ambiente pronto para carregar dados com Pandas.
- **Observação:**
    - Gráficos com estilos otimizados para visualização.

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors
import seaborn as sns
import sqlalchemy as sa
from IPython.display import display

# Configuração visual
plt.style.use('seaborn-v0_8')
sns.set_theme(context="notebook", style="whitegrid")

print("✅ Ambiente Pandas carregado com sucesso!")

ModuleNotFoundError: No module named 'sqlalchemy'

### 1.2 Conexão com o Banco de Dados PostgreSQL (One Big Table)
- **Objetivo:**
    - Conectar ao PostgreSQL e carregar a One Big Table `silver.acd` com Pandas/SQLAlchemy.
- **Ações:**
    - Define credenciais e URL de conexão.
    - Lê a tabela completa para um DataFrame Pandas.
- **Saída:**
    - DataFrame `df_acidentes` com dados da camada SILVER em estrutura normalizada.

In [ ]:
HOST = os.getenv("DB_HOST", "localhost")
PORT = int(os.getenv("DB_PORT", 5432))
DB = os.getenv("DB_NAME", "acidentes_aereos")
USER = os.getenv("DB_USER", "postgres")
PASSWORD = os.getenv("DB_PASSWORD", "postgres")

print(f"Tentando ler a One Big Table 'silver.acd' do banco de dados via Pandas...")

try:
    engine = sa.create_engine(
        f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}"
    )
    df_acidentes = pd.read_sql_query("SELECT * FROM silver.acd", con=engine)
    print("✅ Leitura concluída com sucesso!")
    
    print("\nSchema (dtypes):")
    print(df_acidentes.dtypes)
    
    print("\nAmostra dos dados (formato de tabela):")
    display(df_acidentes.head())
    
    print(f"\nTotal de registros: {len(df_acidentes)}")
except Exception as e:
    print("❌ A conexão falhou. Verifique se o PostgreSQL está rodando e as credenciais.")
    print(f"Erro técnico: {e}")

---
## 2. Gráficos e Análises

### 2.1 Evolução Temporal de Acidentes por Ano

- **Objetivo do gráfico:** 
    - Visualizar a tendência de acidentes ao longo dos anos.
- **Dados utilizados:** 
    - Coluna `Ano` para agrupamento temporal.
- **Ações principais:**
    - Agrupa por ano e conta ocorrências.
    - Converte para Pandas e plota linha temporal.
- **Saída:**
    - Gráfico de linha mostrando evolução temporal.
- **Insight esperado:**
    - Identificar se houve melhorias na segurança ao longo do tempo.

In [ ]:
# Agregar acidentes por ano (Pandas)
acidentes_por_ano = (
    df_acidentes.groupby("ano").size().reset_index(name="Total_Acidentes")
    .sort_values("ano")
)

# Plotar
plt.figure(figsize=(14, 6))
plt.plot(acidentes_por_ano['ano'], acidentes_por_ano['Total_Acidentes'], marker='o', linewidth=2.5, markersize=8, color='#2E86AB')
plt.fill_between(acidentes_por_ano['ano'], acidentes_por_ano['Total_Acidentes'], alpha=0.3, color='#2E86AB')
plt.title('Evolução de Acidentes Aéreos por Ano', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Ano', fontsize=12)
plt.ylabel('Quantidade de Acidentes', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

ano_max_row = acidentes_por_ano.loc[acidentes_por_ano['Total_Acidentes'].idxmax()]
print(f"Ano com mais acidentes: {int(ano_max_row['ano'])} ({int(ano_max_row['Total_Acidentes'])} acidentes)")

### 2.2 Top 10 Estados com Mais Acidentes

- **Objetivo do gráfico:** 
    - Identificar regiões geográficas com maior concentração de acidentes.
- **Dados utilizados:** 
    - Coluna `UF da Ocorrencia`.
- **Ações principais:**
    - Agrupa por UF e ordena por quantidade.
- **Saída:**
    - Gráfico de barras horizontal com top 10.
- **Insight esperado:**
    - Correlacionar com densidade de tráfego aéreo e infraestrutura.

In [ ]:
# Agregar por UF
acidentes_por_uf = df_acidentes.groupBy("uf").agg(
    count("*").alias("Total_Acidentes")
).orderBy(col("Total_Acidentes").desc()).limit(10)

df_uf = acidentes_por_uf.toPandas()

# Plotar
plt.figure(figsize=(14, 8))
ax = sns.barplot(x='Total_Acidentes', y='uf', data=df_uf, orient='h', palette='viridis')

for i, v in enumerate(df_uf['Total_Acidentes']):
    ax.text(v + 5, i, str(v), va='center', fontsize=10, fontweight='bold')

plt.title('Top 10 Estados com Mais Acidentes Aéreos', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Quantidade de Acidentes', fontsize=12)
plt.ylabel('Estado (UF)', fontsize=12)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Estado líder: {df_uf.iloc[0]['uf']} com {df_uf.iloc[0]['Total_Acidentes']} acidentes")

### 2.3 Distribuição de Severidade dos Acidentes

- **Objetivo do gráfico:** 
    - Visualizar proporção de acidentes por nível de severidade.
- **Dados utilizados:** 
    - Coluna `Nivel_Severidade` (CRÍTICO, GRAVE, MODERADO, LEVE).
- **Ações principais:**
    - Agrupa por severidade e conta.
- **Saída:**
    - Gráfico de pizza mostrando distribuição percentual.
- **Insight esperado:**
    - Avaliar proporção de acidentes graves vs leves.

In [ ]:
# Agregar por severidade
acidentes_por_severidade = df_acidentes.groupBy("nvl_sev").agg(
    count("*").alias("Total_Acidentes")
).orderBy(col("Total_Acidentes").desc())

df_severidade = acidentes_por_severidade.toPandas()

# Plotar
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Pizza
colors_sev = ['#FF6B6B', '#FFA500', '#FFD700', '#4ECDC4']
ax1.pie(df_severidade['Total_Acidentes'], labels=df_severidade['nvl_sev'], 
       autopct='%1.1f%%', startangle=90, colors=colors_sev, textprops={'fontsize': 12})
ax1.set_title('Distribuição de Acidentes por Nível de Severidade', fontsize=14, fontweight='bold')

# Barras
ax2.barh(df_severidade['nvl_sev'], df_severidade['Total_Acidentes'], color=colors_sev, edgecolor='black')
ax2.set_title('Quantidade por Nível de Severidade', fontsize=14, fontweight='bold')
ax2.set_xlabel('Quantidade de Acidentes', fontsize=12)
ax2.grid(axis='x', alpha=0.3)

for i, v in enumerate(df_severidade['Total_Acidentes']):
    ax2.text(v + 10, i, str(v), va='center', fontsize=11)

plt.tight_layout()
plt.show()

print(df_severidade)

### 2.4 Top 15 Tipos de Ocorrências Mais Frequentes

- **Objetivo do gráfico:** 
    - Identificar os tipos de acidentes mais comuns.
- **Dados utilizados:** 
    - Coluna `Tipo de Ocorrencia`.
- **Ações principais:**
    - Agrupa e ordena por frequência.
- **Saída:**
    - Gráfico de barras horizontal.
- **Insight esperado:**
    - Direcionar esforços de prevenção aos tipos mais recorrentes.

In [ ]:
# Agregar por tipo de ocorrência
tipos_ocorrencia = df_acidentes.groupBy("tpo_ocr").agg(
    count("*").alias("Total_Ocorrencias")
).orderBy(col("Total_Ocorrencias").desc()).limit(15)

df_tipos = tipos_ocorrencia.toPandas()

# Plotar
plt.figure(figsize=(14, 8))
ax = sns.barplot(x='Total_Ocorrencias', y='tpo_ocr', data=df_tipos, orient='h', palette='rocket', edgecolor='black')

for i, v in enumerate(df_tipos['Total_Ocorrencias']):
    ax.text(v + 2, i, str(v), va='center', fontsize=10)

plt.title('Top 15 Tipos de Ocorrências Mais Frequentes', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Quantidade de Ocorrências', fontsize=12)
plt.ylabel('Tipo de Ocorrência', fontsize=12)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Tipo mais comum: {df_tipos.iloc[0]['tpo_ocr']} ({df_tipos.iloc[0]['Total_Ocorrencias']} ocorrências)")

### 2.5 Média de Fatalidades por Tipo de Ocorrência (Top 15)

- **Objetivo do gráfico:** 
    - Identificar quais tipos de acidentes são mais letais.
- **Dados utilizados:** 
    - Colunas `Tipo de Ocorrencia` e `Total de Fatalidades no Acidente`.
- **Ações principais:**
    - Calcula média de fatalidades por tipo.
    - Filtra tipos com pelo menos 5 ocorrências.
- **Saída:**
    - Gráfico de barras horizontal ordenado por letalidade.
- **Insight esperado:**
    - Priorizar prevenção de tipos com alta média de fatalidades.

In [ ]:
# Calcular média de fatalidades por tipo
fatalidades_por_tipo = df_acidentes.groupBy("tpo_ocr").agg(
    avg("ttl_fat").alias("Media_Fatalidades"),
    count("*").alias("Total_Ocorrencias")
).filter(col("Total_Ocorrencias") >= 5).orderBy(col("Media_Fatalidades").desc()).limit(15)

df_fatalidades = fatalidades_por_tipo.toPandas()

# Plotar
plt.figure(figsize=(14, 8))
colors_fat = ['#FF6B6B' if x > df_fatalidades['Media_Fatalidades'].median() else '#4ECDC4' 
              for x in df_fatalidades['Media_Fatalidades']]

ax = plt.barh(df_fatalidades['tpo_ocr'], df_fatalidades['Media_Fatalidades'], 
              color=colors_fat, edgecolor='black')

for i, v in enumerate(df_fatalidades['Media_Fatalidades']):
    plt.text(v + 0.05, i, f"{v:.2f}", va='center', fontsize=10)

plt.title('Média de Fatalidades por Tipo de Ocorrência (Top 15)', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Média de Fatalidades por Acidente', fontsize=12)
plt.ylabel('Tipo de Ocorrência', fontsize=12)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Tipo mais letal: {df_fatalidades.iloc[0]['tpo_ocr']} (média de {df_fatalidades.iloc[0]['Media_Fatalidades']:.2f} fatalidades)")

### 2.6 Fases de Operação Mais Críticas

- **Objetivo do gráfico:** 
    - Identificar em quais fases da operação ocorrem mais acidentes.
- **Dados utilizados:** 
    - Coluna `Fase de Operacao da Aeronave`.
- **Ações principais:**
    - Agrupa por fase e conta ocorrências.
- **Saída:**
    - Gráfico de barras horizontal com top 10.
- **Insight esperado:**
    - Focar treinamento e protocolos nas fases mais arriscadas.

In [ ]:
# Agregar por fase de operação
fases_operacao = df_acidentes.groupBy("fse_ope").agg(
    count("*").alias("Total_Acidentes")
).orderBy(col("Total_Acidentes").desc()).limit(10)

df_fases = fases_operacao.toPandas()

# Plotar
plt.figure(figsize=(14, 8))
ax = sns.barplot(x='Total_Acidentes', y='fse_ope', data=df_fases, 
                orient='h', palette='mako', edgecolor='black')

for i, v in enumerate(df_fases['Total_Acidentes']):
    ax.text(v + 5, i, str(v), va='center', fontsize=10, fontweight='bold')

plt.title('Top 10 Fases de Operação com Mais Acidentes', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Quantidade de Acidentes', fontsize=12)
plt.ylabel('Fase de Operação', fontsize=12)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Fase mais crítica: {df_fases.iloc[0]['fse_ope']} ({df_fases.iloc[0]['Total_Acidentes']} acidentes)")

### 2.7 Distribuição Mensal de Acidentes

- **Objetivo do gráfico:** 
    - Identificar padrões sazonais de acidentes.
- **Dados utilizados:** 
    - Coluna `Mes`.
- **Ações principais:**
    - Agrupa por mês e conta ocorrências.
- **Saída:**
    - Gráfico de barras mostrando distribuição ao longo do ano.
- **Insight esperado:**
    - Correlacionar com condições climáticas sazonais.

In [ ]:
# Agregar por mês
acidentes_por_mes = df_acidentes.groupBy("mes").agg(
    count("*").alias("Total_Acidentes")
).orderBy("mes")

df_mes = acidentes_por_mes.toPandas()

# Nomes dos meses
meses_nomes = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
df_mes['Mes_Nome'] = df_mes['mes'].apply(lambda x: meses_nomes[int(x)-1] if pd.notna(x) and 1 <= x <= 12 else 'N/A')

# Plotar
plt.figure(figsize=(14, 6))
colors_mes = plt.cm.viridis(np.linspace(0, 1, 12))
plt.bar(df_mes['Mes_Nome'], df_mes['Total_Acidentes'], color=colors_mes, edgecolor='black')

plt.title('Distribuição Mensal de Acidentes Aéreos', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Mês', fontsize=12)
plt.ylabel('Quantidade de Acidentes', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

mes_pico = df_mes.loc[df_mes['Total_Acidentes'].idxmax()]
print(f"Mês com mais acidentes: {mes_pico['Mes_Nome']} ({int(mes_pico['Total_Acidentes'])} acidentes)")

---
## 3. Estatísticas Descritivas

### 3.1 Resumo Geral dos Dados

In [ ]:
print("\n" + "="*70)
print("ESTATÍSTICAS GERAIS DA ONE BIG TABLE (SILVER)")
print("="*70)

# Totais
total_acidentes = df_acidentes.count()
total_fatalidades = df_acidentes.agg(sum("ttl_fat")).collect()[0][0]
total_recomendacoes = df_acidentes.agg(sum("ttl_rec")).collect()[0][0]

print(f"\nTotal de acidentes registrados: {total_acidentes:,}")
print(f"Total de fatalidades: {int(total_fatalidades):,}")
print(f"Total de recomendações emitidas: {int(total_recomendacoes):,}")

# Médias
media_fatalidades = df_acidentes.agg(avg("ttl_fat")).collect()[0][0]
media_recomendacoes = df_acidentes.agg(avg("ttl_rec")).collect()[0][0]

print(f"\nMédia de fatalidades por acidente: {media_fatalidades:.2f}")
print(f"Média de recomendações por acidente: {media_recomendacoes:.2f}")

# Intervalos
ano_min = df_acidentes.agg(min("ano")).collect()[0][0]
ano_max = df_acidentes.agg(max("ano")).collect()[0][0]

print(f"\nPeríodo de cobertura: {int(ano_min)} a {int(ano_max)}")
print(f"Total de anos: {int(ano_max - ano_min + 1)}")

# Estados distintos
total_ufs = df_acidentes.select("uf").distinct().count()
print(f"\nEstados com acidentes registrados: {total_ufs}")

print("\n" + "="*70)

---
## 4. Finalização

### 4.1 Encerrar Sessão Spark

In [ ]:
print("\n🚀 Análise da camada SILVER concluída com sucesso!")
print("Encerrando a sessão Spark...")
spark.stop()
print("✅ Sessão encerrada.")